# Simple Scenario Analysis

This notebook demonstrates how to access and use the functionalities of **Scenario Analysis** which are part of our **Yiedbook** module within LSEG Financial Analytics SDK. The notebook outlines the most simple method of using performing scenario analysis, where only the required set of arguments are present.

The main goal of this notebook is to explore and display an approach in fetching Scenario Analysis information in it's simplest form by using any of 4 available methods:

1. Synchronous POST
2. Synchronous GET
3. Asynchronous POST
4. Asynchronous GET

## Imports

Import the following necessary modules:

- *request_scenario_calculation_sync* - main Synchronous POST method
- *ScenarioCalcGlobalSettings* - Class wrapper for Global Settings 
- *ScenarioCalcInput* - Class wrapper for Input parameters
- *Scenario* - Class wrapper for defining various Scenarios
- *json* - for display purposes  

In [1]:
from lseg_analytics.pricing.yield_book_rest import (
        request_scenario_calculation_sync,
        request_get_scen_calc_sys_scen_sync,
        request_scenario_calculation_async,
        request_get_scen_calc_sys_scen_async,
        get_result,
        Volatility,
        StructureNote,
        CurveTypeAndCurrency,
        LossSettings,
        Vector,
        MonthRatePair,
        RestPrepaySettings,
        ScenarioCalcGlobalSettings,
        Scenario,
        ScenarioDefinition,
        SystemScenario,
        UserScenario,
        ApimCurveShift,
        CurveMultiShift,
        ScenAbsoluteCurvePoint,
        ScenarioVolatility,
        SwaptionVolatility,
        SwaptionVolItem,
        CapVolatility,
        CapVolItem,
        ScenarioCalcInput,
        SettlementInfo,
        PricingScenario,
        CustomScenario,
        CmbsPrepayment,
        Balloon,
        HorizonInfo,
        ScenarioCalcFloaterSettings,
        HecmSettings,
        ScenCalcExtraSettings,
        Partials,
)
import json as js
import time

## Data Preparation

Cash Flow Post methods consume three input arguments: 
- Global settings 
    - settings applicable in overall Scenario Analysis
- Inputs
    - separate parameters to describe and enable specific SA information
- Scenario
    - individual definition of Scenarios used for analysis

In [2]:
# request_scenario_calculation_async
global_settings = ScenarioCalcGlobalSettings(
            pricing_date="2025-01-01",
        )

scenario = Scenario(
            scenario_id="ScenID1",
            definition=ScenarioDefinition(
                system_scenario=SystemScenario(name="BEARFLAT100")
            ),
        )

input = ScenarioCalcInput(
            identifier="US742718AV11",
            id_type="ISIN",
            curve=CurveTypeAndCurrency(
                curve_type="GVT",
                currency="USD",
            ),
            settlement_info=SettlementInfo(
                level="100",
            ),
            horizon_info=[
                HorizonInfo(
                    scenario_id="ScenID1",
                    level="100",
                )
            ],
            horizon_py_method="OAS",
        )

## Synchronous POST Data retrieval

Once the input arguments are ready, they can be passed through the main method for data retrieval

In [3]:
# Execute Post sync request with prepared inputs
sa_sync_post_response = request_scenario_calculation_sync(
                            global_settings=global_settings,
                            scenarios=[scenario],
                            input=[input],
                        )

## Synchronous GET Data retrieval

As opposed to the POST method, the GET method has the following mandatory arguments:
- identifier
- horizon py method
- curve type
- pricing date
- horizon level 
- existing scenario name. 

The remaining arguments are optional. 

In [4]:
# Formulate and execute the get request
sa_sync_get_response = request_get_scen_calc_sys_scen_sync(
            id='US742718AV11',
            h_py_method="OAS",
            curve_type="GVT",
            pricing_date="2025-01-01",
            level="100",
            scenario="/sys/scenario/Par/50"
        )

## Asynchronous POST Data retrieval

The main difference between the Sync and Async approach is that Async methods allow execution of code on the user's side while waiting for a response. Once execution is complete, the user can 'collect' the results whenever they need them.

So this approach has 2 steps:
1. Provide input data to execution method
2. Use the request_id received in step 1 to retrieve the result

In [5]:
# Request bond CF with async post
sa_async_post_response = request_scenario_calculation_async(
                            global_settings=global_settings,
                            scenarios=[scenario],
                            input=[input],
                        )

attempt = 1

while attempt < 10:

    from lseg_analytics.core.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_post_results_response = get_result(request_id_parameter=sa_async_post_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + sa_async_post_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{sa_async_post_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

[ERROR]	[2025-12-03 12:44:58,558]	[MainThread]	[lseg_analytics.pricing.yield_book_rest]	[_functions.py:10384]	 Error get_result.
[ERROR]	[2025-12-03 12:44:58,560]	[MainThread]	[lseg_analytics.core]	[exceptions.py:78]	 Exception: Resource not found: code=404 {'meta': {'requestId': 'R-150643', 'status': 'RUNNING', 'responseType': 'SCENARIO_CALC', 'message': 'The result is not ready yet for requestID:R-150643', 'interval': 0.1}}
Attempt 1 resulted in error retrieving results from:R-150643


## Asynchronous GET Data retrieval

Difference between Sync and Async can be found in description above.

As Async execution is in queue-form it's not necessarily instantly finished. It can take time between submission of request and execution, and results are not in real-time.

Hence we present also the form of iterative results-request repetition to fetch the results as soon as they are ready.

**NOTE:** Alternative to this is wrapping the request in the job, and then using a method to check the job status. See the fundamentals notebook for information on jobs.

In [6]:
# Formulate and execute the get request by using instrument ID, Par_amount and job in which the calculation will be done
sa_async_get_response = request_get_scen_calc_sys_scen_async(
            id='US742718AV11',
            scenario="/sys/scenario/Par/50",
            pricing_date="2025-01-01",
            curve_type="GVT",
            h_py_method="OAS",
            h_level="100",
            level="100"
        )

# Due to async nature, code Will perform the fetch 10 times, as result is not always ready instantly, with 3 second lapse between attempts
attempt = 1

while attempt < 10:

    from lseg_analytics.core.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_get_results_response = get_result(request_id_parameter=sa_async_get_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + sa_async_get_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{sa_async_get_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

[ERROR]	[2025-12-03 12:45:01,964]	[MainThread]	[lseg_analytics.pricing.yield_book_rest]	[_functions.py:10384]	 Error get_result.
[ERROR]	[2025-12-03 12:45:01,965]	[MainThread]	[lseg_analytics.core]	[exceptions.py:78]	 Exception: Resource not found: code=404 {'meta': {'requestId': 'R-150644', 'status': 'RUNNING', 'responseType': 'SCENARIO_CALC', 'message': 'The result is not ready yet for requestID:R-150644', 'interval': 0.1}}
Attempt 1 resulted in error retrieving results from:R-150644


## Display results

Results data is structured in JSON format so it can be easily displayed using print function.

### Synchronous POST results

In [7]:
# Print output in JSON format
print(js.dumps(obj=sa_sync_post_response, indent=4))

{
    "meta": {
        "status": "DONE",
        "requestId": "R-150641",
        "timeStamp": "2025-12-03T07:44:57Z",
        "responseType": "SCENARIO_CALC",
        "resultsStatus": "ALL"
    },
    "results": [
        {
            "isin": "US742718AV11",
            "cusip": "742718AV1",
            "ticker": "PG",
            "scenario": {
                "horizon": [
                    {
                        "oas": 100.0,
                        "wal": 4.8139,
                        "price": 104.89765,
                        "yield": 6.7865,
                        "balance": 1.0,
                        "pylevel": "100.000000",
                        "duration": 3.9209,
                        "fullPrice": 106.386538,
                        "returnCode": 0,
                        "scenarioID": "ScenID1",
                        "spreadDV01": 0.0,
                        "volatility": 16.0,
                        "actualPrice": 104.898,
                        "gross

### Synchronous GET results

In [8]:
# Print output in JSON format
print(js.dumps(obj=sa_sync_get_response, indent=4))

{
    "data": {
        "isin": "US742718AV11",
        "cusip": "742718AV1",
        "ticker": "PG",
        "scenario": {
            "horizon": [
                {
                    "oas": 361.951,
                    "wal": 4.8139,
                    "price": 98.053324,
                    "yield": 8.4961,
                    "balance": 1.0,
                    "duration": 3.8584,
                    "fullPrice": 99.542213,
                    "returnCode": 0,
                    "scenarioID": "/sys/scenario/Par/50",
                    "spreadDV01": 0.0,
                    "volatility": 16.0,
                    "actualPrice": 98.053,
                    "grossSpread": 361.3423,
                    "horizonDays": 0,
                    "marketValue": 99.542213,
                    "optionValue": 0.0,
                    "totalReturn": -1.91811763,
                    "dollarReturn": -1.94667627,
                    "convexityCost": 0.0,
                    "nominalSpread": 361

### Asynchronous POST results

In [9]:
# Print output in JSON format
print(js.dumps(obj=async_post_results_response, indent=4))

{
    "meta": {
        "status": "DONE",
        "requestId": "R-150643",
        "timeStamp": "2025-12-03T07:44:58Z",
        "responseType": "SCENARIO_CALC",
        "resultsStatus": "ALL"
    },
    "results": [
        {
            "isin": "US742718AV11",
            "cusip": "742718AV1",
            "ticker": "PG",
            "scenario": {
                "horizon": [
                    {
                        "oas": 100.0,
                        "wal": 4.8139,
                        "price": 104.89765,
                        "yield": 6.7865,
                        "balance": 1.0,
                        "pylevel": "100.000000",
                        "duration": 3.9209,
                        "fullPrice": 106.386538,
                        "returnCode": 0,
                        "scenarioID": "ScenID1",
                        "spreadDV01": 0.0,
                        "volatility": 16.0,
                        "actualPrice": 104.898,
                        "gross

### Asynchronous GET results

In [10]:
# Print output in JSON format
print(js.dumps(obj=async_get_results_response, indent=4))

{
    "data": {
        "isin": "US742718AV11",
        "cusip": "742718AV1",
        "ticker": "PG",
        "scenario": {
            "horizon": [
                {
                    "oas": 100.0,
                    "wal": 4.8139,
                    "price": 108.774734,
                    "yield": 5.8775,
                    "balance": 1.0,
                    "pylevel": "100.000000",
                    "duration": 3.9542,
                    "fullPrice": 110.263623,
                    "returnCode": 0,
                    "scenarioID": "/sys/scenario/Par/50",
                    "spreadDV01": 0.0,
                    "volatility": 16.0,
                    "actualPrice": 108.775,
                    "grossSpread": 99.4751,
                    "horizonDays": 0,
                    "marketValue": 110.263623,
                    "optionValue": 0.0,
                    "totalReturn": 8.64600454,
                    "dollarReturn": 8.77473393,
                    "convexityCost": 0